In [8]:
import random

# Define a card deck
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']
rank_values = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'Jack': 11, 'Queen': 12, 'King': 13, 'Ace': 14}
deck = [(rank, suit) for suit in suits for rank in ranks]

def shuffle_deck(deck):
    """Shuffle the deck of cards."""
    random.shuffle(deck)
    return deck

def deal_hand(deck):
    """Deal a three-card hand."""
    return [deck.pop() for _ in range(3)]

def hand_rank(hand):
    """Evaluate the rank of a poker hand."""
    ranks = sorted([rank_values[card[0]] for card in hand], reverse=True)
    suits = [card[1] for card in hand]

    is_flush = len(set(suits)) == 1
    is_straight = ranks[0] - ranks[2] == 2 and len(set(ranks)) == 3

    if is_straight and is_flush:
        return (9, ranks)
    elif len(set(ranks)) == 1:
        return (8, ranks)
    elif is_straight:
        return (7, ranks)
    elif is_flush:
        return (6, ranks)
    elif len(set(ranks)) == 2:
        return (5, ranks)
    else:
        return (4, ranks)

def dealer_plays(hand):
    """Check if the dealer's hand qualifies (Queen high or better)."""
    return sorted([rank_values[card[0]] for card in hand], reverse=True)[0] >= rank_values['Queen']

def compare_hands(player_hand, dealer_hand):
    """Compare player's and dealer's hands to determine the winner."""
    if not dealer_plays(dealer_hand):
        return "player"

    player_rank = hand_rank(player_hand)
    dealer_rank = hand_rank(dealer_hand)

    if player_rank > dealer_rank:
        return "player"
    elif player_rank < dealer_rank:
        return "dealer"
    else:
        # tie
        return "player"

# Simulate a game
shuffled_deck = shuffle_deck(deck[:])
player_hand = deal_hand(shuffled_deck)
dealer_hand = deal_hand(shuffled_deck)

print("Player's hand:", player_hand)
print("Dealer's hand:", dealer_hand)
print(compare_hands(player_hand, dealer_hand))


Player's hand: [('2', 'Spades'), ('Ace', 'Spades'), ('6', 'Hearts')]
Dealer's hand: [('6', 'Diamonds'), ('3', 'Spades'), ('6', 'Spades')]
player


In [5]:
import csv 

def simulate_games(n):
    """Simulate n games and generate training data."""
    data = []

    for _ in range(n):
        shuffled_deck = shuffle_deck(deck[:])
        player_hand = deal_hand(shuffled_deck)
        dealer_hand = deal_hand(shuffled_deck)

        player_hand_rank = hand_rank(player_hand)
        dealer_qualifies = dealer_plays(dealer_hand)
        game_result = compare_hands(player_hand, dealer_hand)

        # Record data points
        record = {
            'player_card_1_rank': player_hand[0][0],
            'player_card_1_suit': player_hand[0][1],
            'player_card_2_rank': player_hand[1][0],
            'player_card_2_suit': player_hand[1][1],
            'player_card_3_rank': player_hand[2][0],
            'player_card_3_suit': player_hand[2][1],
            'player_hand_rank': player_hand_rank[0],
            'dealer_qualifies': dealer_qualifies,
            'game_result': game_result
        }
        data.append(record)

    return data

def write_to_csv(data, filename):
    """Write the generated data to a CSV file."""
    keys = data[0].keys()
    with open(filename, 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

# Simulate 1000 games and write the data to a CSV file
training_data = simulate_games(1000)
write_to_csv(training_data, 'three_card_poker_training_data.csv')

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Load the data
data = pd.read_csv('three_card_poker_training_data.csv')

# Initialize a dictionary to store label encoders
label_encoders = {}

# Convert categorical features to numerical values using Label Encoders
for column in ['player_card_1_rank', 'player_card_1_suit', 'player_card_2_rank', 'player_card_2_suit', 'player_card_3_rank', 'player_card_3_suit']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Save each encoder
for column, le in label_encoders.items():
    joblib.dump(le, f'{column}_encoder.pkl')

# Convert target variable 'game_result' to numerical values
data['game_result'] = data['game_result'].map({'Player wins!': 1, 'Dealer wins!': 0, 'It\'s a tie!': 2})

# Split the data into features and target variable
X = data.drop('game_result', axis=1)
y = data['game_result']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Choose and train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Save the trained model
joblib.dump(model, 'poker_model.pkl')

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


ValueError: Input y contains NaN.